In [5]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset,ConcatDataset
from sklearn.metrics import f1_score
import numpy as np

from torch.autograd import Variable

# Configure Random seeds
torch.manual_seed(777)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(777)


In [6]:
# Define custom dataset - train set
class trainDataset(Dataset):
    
    def __init__(self):


        self.time_train_x = torch.load('time_train_X.pt',map_location=torch.device('cpu'))
        self.static_train_x = torch.load('static_train_X.pt',map_location=torch.device('cpu'))
        
        self.train_y = torch.load('train_y.pt', map_location=torch.device('cpu'))

        self.len= len(self.time_train_x)
 
    def __getitem__(self,index):
        return self.time_train_x[index], self.static_train_x[index],  self.train_y[index]
    
    def __len__(self):
        return self.len

# Define custom dataset - test set    
class testDataset(Dataset):
    
    def __init__(self):


        self.time_test_x = torch.load('time_test_X.pt',map_location=torch.device('cpu'))
        self.static_test_x = torch.load('static_test_X.pt',map_location=torch.device('cpu'))
        
        self.test_y = torch.load('test_y.pt', map_location=torch.device('cpu'))

        self.len= len(self.time_test_x)
 
    def __getitem__(self,index):
        return self.time_test_x[index], self.static_test_x,  self.test_y[index]
    
    def __len__(self):
        return self.len

    
# Define custom dataset - test set    
class valDataset(Dataset):
    
    def __init__(self):


        self.time_val_x = torch.load('time_val_X.pt',map_location=torch.device('cpu'))
        self.static_val_x = torch.load('static_val_X.pt',map_location=torch.device('cpu'))
        
        self.val_y = torch.load('val_y.pt', map_location=torch.device('cpu'))

        self.len= len(self.time_val_x)
 
    def __getitem__(self,index):
        return self.time_val_x[index], self.static_val_x[index],  self.val_y[index]
    
    def __len__(self):
        return self.len

# Hyper parameter setting
batch_size = 100

# Create datsets and dataloader
trainset = trainDataset()
testset = testDataset()
valset = valDataset()
train_loader =DataLoader(trainset, batch_size = batch_size, shuffle = True)
val_loader = DataLoader(valset, batch_size = batch_size, shuffle = False)
test_loader= DataLoader(testset, batch_size = batch_size, shuffle = False)

In [29]:
class modelFinal(nn.Module):
    def __init__(self, input_size=35, hidden_size=10, num_layers=1, output_size=1):
        super(modelFinal, self).__init__()
        self.num_layers = num_layers 
        self.input_size = input_size 
        self.hidden_size = hidden_size 
        self.output_size = output_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) 
        self.fc1 =  nn.Linear(hidden_size, 32,bias=True) 
        self.fc2 = nn.Linear(7, 32,bias=True) 
        
        
        self.fc3 = nn.Linear(32 + 32, 32,bias=True)
        self.fc4 = nn.Linear(32,1,bias=True)

    
        
        self.Sigmoid = torch.nn.Sigmoid()
        self.relu = nn.ReLU()
    
    
        self.dropout = torch.nn.Dropout(p=0.2)
    def forward(self,x_time, x_static):
        h_0 = Variable(torch.zeros(self.num_layers, x_time.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x_time.size(0), self.hidden_size)) #internal state
       
        output, (hn, cn) = self.lstm(x_time, (h_0, c_0))
        hn = hn.view(-1, self.hidden_size) 
        out_time = self.relu(hn)
        out_time = self.fc1(out_time) 
        out_time = self.relu(out_time)
        
        out_static = self.fc2(x_static)
        out_static = self.relu(out_static)
        
        out = torch.cat((out_time, out_static), dim=1)
       
        out = (self.fc3(out))
        out = (self.relu(out))
        out = (self.fc4(out))

        y_pred = self.Sigmoid(out)
        return y_pred
 

In [39]:
model = modelFinal()
loss_function = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)


params = list(model.parameters())
print("The number of parameters:", sum([p.numel() for p in model.parameters() if p.requires_grad]), "elements")


The number of parameters: 4601 elements


In [37]:
epochs = 110

# train the model
valid_loss_min = int(1e9)
for epoch in range(epochs):
    
    train_loss=0.0
    valid_loss=0.0
    
    # Train 
    
    model.train()
    # loop for ever batch set in train dataset loader
    for i, (data_time, data_static, target) in enumerate(train_loader):
        

        # initialize optimizer
        optimizer.zero_grad()
        
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_size),
                        torch.zeros(1, 1, model.hidden_size))
        
        
        # train
        y_pred = model(data_time, data_static)
        y_pred = y_pred.squeeze()
     
        # caculate batch loss
        loss = loss_function(y_pred, target)
        
        # update
        loss.backward(retain_graph=True)
        optimizer.step()
    
        # Record train loss
        train_loss += loss.item() 
    
    # Validate 
    
    model.eval()
    # loop for ever batch set in test dataset loader
    for i, (data_time, data_static, target) in enumerate(val_loader):

        y_pred = model(data_time, data_static)
        y_pred = y_pred.squeeze()
        
        # calculate the batch loss
        loss = loss_function(y_pred, target)
        
        # calculate validataion loss
        valid_loss += loss.item()
        
        
    # calculate average losses
    train_loss = train_loss / len(train_loader)
    valid_loss = valid_loss / len(val_loader)
    
    # print training/validation results
    print('Epoch: {}     Training Loss: {}    Validation Loss: {}'.format(epoch,train_loss, valid_loss))
    
    # if trained model perform best vaildation loss, then save it to the checkpoint
    if valid_loss <= valid_loss_min:
        print('Decreased Validation Loss ({} ==> {})   < model saved >'.format(valid_loss_min,valid_loss))
        
        checkpoint = {
            'state_dict': model.state_dict(),
        }
        torch.save(checkpoint, './bestModel.pt')
        valid_loss_min = valid_loss


        

    

Epoch: 0     Training Loss: 0.13919957129567756    Validation Loss: 0.15206441411473712
Decreased Validation Loss (1000000000 ==> 0.15206441411473712)   < model saved >
Epoch: 1     Training Loss: 0.1377117544833847    Validation Loss: 0.14997093336748296
Decreased Validation Loss (0.15206441411473712 ==> 0.14997093336748296)   < model saved >
Epoch: 2     Training Loss: 0.13690540129351741    Validation Loss: 0.14979648510696458
Decreased Validation Loss (0.14997093336748296 ==> 0.14979648510696458)   < model saved >
Epoch: 3     Training Loss: 0.13725323383383742    Validation Loss: 0.1533356993535503
Epoch: 4     Training Loss: 0.13754702576636846    Validation Loss: 0.15141493582823237
Epoch: 5     Training Loss: 0.1377350726418276    Validation Loss: 0.14893470991586075
Decreased Validation Loss (0.14979648510696458 ==> 0.14893470991586075)   < model saved >
Epoch: 6     Training Loss: 0.13741017941240707    Validation Loss: 0.15146696353789235
Epoch: 7     Training Loss: 0.137847

KeyboardInterrupt: 

In [38]:
# Load best performance model
checkpoint = torch.load('./bestModel.pt')
trained_model = modelFinal()


trained_model.load_state_dict(checkpoint['state_dict'])

# evaluate
trained_model.eval()
with torch.no_grad():
    
    # load test dataset

    time_test_x = torch.load('time_test_X.pt',map_location=torch.device('cpu'))
    static_test_x = torch.load('static_test_X.pt',map_location=torch.device('cpu'))
    test_y = torch.load('test_y.pt', map_location=torch.device('cpu'))
    # evaluate using weighted f1 score.
    y_pred = trained_model(time_test_x, static_test_x)
    yp = y_pred.detach().numpy()
    yp = [1.0 if x > 0.5 else 0.0 for x in yp]

    print("weighted F1:", f1_score(test_y, yp, average='weighted'))

weighted F1: 0.9421980250140295
